In [1]:
import os
import shutil
import sqlite3
from typing import List

import qrcode
import pandas as pd
from tqdm import tqdm
from utils.batch_processing import zip_batch

# ReportLab
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics

In [2]:
conn = sqlite3.connect("../data/database.sqlite")
cur = conn.cursor()

data = pd.read_sql("SELECT * FROM keys", conn)
data

,encrypted_id,encryption_key,unencrypted_id,date_added,is_redeemed
0,4ULdDFoRq7lJ82K3F2GOb+0cLmPfO32N0rpnK6A01Cw=,b's\xcfS)\x80\x9e\x8d\x1e\xca\xf4b#\xa7\x10[\x...,MT_00001,2025-01-04 13:20:37,0
1,tXRtZrzLDItsOZOCcq4zRUAwSrxWuNOD0cz32PEgpEY=,b'\xf7pm\xce\xa8\xd6\xb6\xc8\xed\xc7[J2\xc8\xf...,MT_00002,2025-01-04 13:20:37,0
2,SSPdFUHEnA7Q5QQgD3sq8la/+UtVYOJdmJkofC6g9xY=,b'\x7f\x9fq\xb1\r\x9f\x1c\x9d\x8a\x14\xb8\x1f2...,MT_00003,2025-01-04 13:20:37,0
3,dpI/ghLK5i+h+UheW8OtMEaF55I9i7l+1/pQkdx+OEM=,b'{\xa8\xda\x83\xb3\xe0\xe7J\xef\xf9Z\xa7\xf33...,MT_00004,2025-01-04 13:20:37,0
4,cuauBQYUV/S+3YqxsERyRFqo0oL2PH+ydR0e/o6P2uw=,b'\x97I\xa7\x1fp)\xb8\xd0\x0eHcW\xf4A\xc7\xbf<...,MT_00005,2025-01-04 13:20:37,0
...,...,...,...,...,...
1997,I+ssnef0wZtqpfH6GKQ5BIg+50o6zcYm9VA1YSveasA=,"b'\x1eC""\x96\xa5[\xe5n\xff\x9e\xd4.\x89\x8b\xd...",MT_01998,2025-01-04 13:20:37,0
1998,QKHjWw2gUTeX6x56YdY93Qi9HhZnjwyyiACN4MrXSto=,b'\x8aZ\x8aSc\x99\xc5T\xd5\xb9J\xad\xe5cfuW&Os...,MT_01999,2025-01-04 13:20:37,0
1999,Ksa7A+14tPVl4HDHEJ0XW8xADNFsetpYqlJy/LEnIz0=,b'\xd7\x0e\xd3\xbaWdX\x96\x1a\x98\x19\xcf\xb2\...,MT_02000,2025-01-04 13:20:37,0
2000,fSJxTknhxXTopJVtRcKbm4fBQXzoaustJzpOozhuwV8=,b'H7\n\xbb\xf3\xd7\x12\xa7\xa9z\x9b\xba_\x80\t...,MT_02001,2025-01-04 13:20:37,0


### Generate QR codes

In [7]:
try:
    if os.path.exists('qr_codes'):
        shutil.rmtree('qr_codes')

    os.makedirs('qr_codes')

    for idx, x in tqdm(data.iterrows(), desc="Generating QR codes", total=len(data)):
        enc_key = x['encryption_key']
        enc_id = x['encrypted_id']
        unenc_id = x['unencrypted_id']
        # Generate the QR code
        qr = qrcode.QRCode(
            version=1,
            error_correction=qrcode.constants.ERROR_CORRECT_L,
            box_size=10,
            border=4,
        )

        qr.add_data(enc_id)
        qr.make(fit=True)

        # Create and save the QR code image
        img = qr.make_image(fill='black', back_color='white')
        img.save(f"qr_codes/{unenc_id}.png")

except Exception as e:
    print(f"{type(e).__name__}: Error in generating QR codes. {e}")

Generating QR codes: 100%|██████████| 2002/2002 [00:12<00:00, 163.83it/s]


### Mass PDF generation

In [3]:
class MealStubPDF(canvas.Canvas):
    """A class to create a PDF with meal stub details, including QR code, ID, and border.
    Methods
    -------
    draw_qr_code(x, y, image_path, width, height)
        Draws a QR code image at the specified location with the given dimensions.
    draw_id(text)
        Draws the ID text at the specified location.
    draw_border(x, y, width, height)
        Draws a border rectangle at the specified location with the given dimensions.
    """
    
    def draw_qr_code(self, x, y, image_path, width, height):
        self.drawImage(image_path, x, y, width, height)

    def draw_id(self, text):
        self.setFont("Helvetica", 9)
        self.drawString(x, y, text)

    def draw_border(self, x, y, width, height, dashed):

        if dashed:
            self.setDash(1, 10)
            
        self.rect(x, y, width, height)

    def draw_admit_line(self, start_x, start_y, end_x, end_y, dashed):
        
        if dashed:
            self.setDash(2, 3)
        
        self.line(start_x, start_y, end_x, end_y)

def cm_to_points(centimeter) -> float:
    """Convert a measurement from centimeters to points.

    Args:
        centimeter (float): The measurement in centimeters to be converted.

    Returns:
        float: The measurement converted to points.
    """
    
    return centimeter * 72 / 2.54

def inch_to_points(inch) -> float:
    """Convert a measurement from inches to points.

    Args:
        inch (float): The measurement in inches to be converted.

    Returns:
        float: The measurement converted to points.
    """

    return inch * 72

In [9]:
def create_A4_batch(
        ids: List[str],
        pdf = MealStubPDF
        ):
    """ Creates a batch of meal stub PDFs in A4 paper size.

    Args:
        ids (List[str]): List of IDs to generate QR codes and meal stubs for.
        pdf (class, optional): PDF generation class. Defaults to MealStubPDF.

    Raises:
        Exception: If there is an error in creating the PDF.

    The function performs the following steps:
        1. Sets up the save directory and filename for the PDF.
        2. Registers the custom font "Night-Driver".
        3. Defines the page size as A4.
        4. Initializes the PDF with the specified filename and page size.
        5. Draws a middle border line on the PDF.
        6. Iterates over the list of IDs to generate QR codes and meal stubs on both the left and right sides of the page.
        7. Draws QR codes, borders, admit lines, brand names, and admit strings for each ID.
        8. Saves the PDF after all stubs are generated.
    """

    try:
        save_directory = "pdfs"
        filename = f"{ids[0]}-{ids[-1]}.pdf"
        full_path = os.path.join(save_directory, filename)
        os.makedirs(save_directory, exist_ok=True)

        font_filename = "nightdriver.ttf"
        font_path = os.path.join("static/fonts", font_filename)
        font_name = "Night-Driver"
        pdfmetrics.registerFont(TTFont(font_name, font_path))

        pagesize = (
            inch_to_points(8.3),
            inch_to_points(13.5) # legal paper size
            ) # A4 Paper size

        meal_stub = pdf(
            filename = full_path,
            pagesize = pagesize # A4 Paper size
            )

        meal_stub.line(
            inch_to_points(4.15),
            inch_to_points(0),
            inch_to_points(4.15),
            inch_to_points(14)
            ) # Middle Border

        box_width = inch_to_points(4.15 - 0.50)
        box_height = inch_to_points(1)

        ids_left = ids[0:13]
        ids_right = ids[13:26]

        for i in range(0, len(ids_left), 1):

            # <Left Side>
            image_directory = "qr_codes"
            image_file_left = f"{ids_left[i]}.png"
            image_path_left = os.path.join(image_directory, image_file_left)

            meal_stub.draw_qr_code(
                x = inch_to_points(0.25),
                y = inch_to_points(i + 0.25),
                image_path = image_path_left,
                width = cm_to_points(2.5),
                height = cm_to_points(2.5)
                )

            meal_stub.draw_border(
                x = inch_to_points(0.25), # Left margin
                y = inch_to_points(i + 0.25), # Bottom margin
                width = box_width,
                height = box_height,
                dashed = True
                )

            meal_stub.draw_admit_line(
                start_x=box_width - inch_to_points(0.25),
                start_y=inch_to_points(i + 0.25),
                end_x=box_width - inch_to_points(0.25),
                end_y=inch_to_points(i + 1.25),
                dashed=True
            )

            # Brand Name
            meal_stub.setFillColor(colors.black)
            meal_stub.setFont("Times-Bold", 12)
            meal_stub.drawString(
                x = inch_to_points(2.1),
                y = inch_to_points(i + 0.95),
                text = "MCES"
            )

            meal_stub.setFillColor(colors.red)
            meal_stub.setFont("Night-Driver", 15)
            meal_stub.drawString(
                x = inch_to_points(1.73),
                y = inch_to_points(i + 0.65),
                text = "BALIK-LANTAW"
            )

            meal_stub.setFillColor(colors.red)
            meal_stub.setFont("Night-Driver", 12)
            meal_stub.drawString(
                x = inch_to_points(2.15),
                y = inch_to_points(i + 0.4),
                text = "2026"
            )

            meal_stub.saveState()
            x = inch_to_points(3.7)
            y = inch_to_points(i + 0.25 + 0.15)
            meal_stub.translate(x, y)
            meal_stub.rotate(90)

            # Admit String
            meal_stub.setFillColor(colors.black)
            meal_stub.setFont("Helvetica", 9)
            meal_stub.drawString(
                x = inch_to_points(-0.055),
                y = inch_to_points(0),
                text = "1 MEAL STUB"
            )

            meal_stub.setFillColor(colors.black)
            meal_stub.drawString(
                x = inch_to_points(0.05),
                y = inch_to_points(2.4),
                text = ids_left[i]
            )

            meal_stub.restoreState()

            # <Right Side>
            image_file_right = f"{ids_right[i]}.png"
            image_path_right = os.path.join(image_directory, image_file_right)

            meal_stub.draw_qr_code(
                x = inch_to_points(4.15 + .25),
                y = inch_to_points(i + 0.25),
                image_path = image_path_right,
                width = cm_to_points(2.5),
                height = cm_to_points(2.5)
                )

            meal_stub.draw_border(
                x = inch_to_points(4.15 + 0.25), # Left margin
                y = inch_to_points(i + 0.25), # Bottom margin
                width = box_width,
                height = box_height,
                dashed = True
            )

            meal_stub.draw_admit_line(
                start_x=inch_to_points(8.3 - 0.75),
                start_y=inch_to_points(i + 0.25),
                end_x=inch_to_points(8.3 - 0.75),
                end_y=inch_to_points(i + 1.25),
                dashed=True
            )

            meal_stub.saveState()
            x = inch_to_points((8.3 / 2) + 3.7)
            y = inch_to_points(i + 0.25 + 0.15)
            meal_stub.translate(x, y)
            meal_stub.rotate(90)

            # Admit String
            meal_stub.setFillColor(colors.black)
            meal_stub.setFont("Helvetica", 9)
            meal_stub.drawString(
                x = inch_to_points(-0.055),
                y = inch_to_points(0),
                text = "1 MEAL STUB"
            )

            meal_stub.setFillColor(colors.black)
            meal_stub.drawString(
                x = inch_to_points(0.05),
                y = inch_to_points(2.4),
                text = ids_right[i]
            )
            meal_stub.restoreState()

            # Brand Name
            meal_stub.setFillColor(colors.black)
            meal_stub.setFont("Times-Bold", 12)
            meal_stub.drawString(
                x = inch_to_points((8.3 / 2) + 2.1),
                y = inch_to_points(i + 0.95),
                text = "MCES"
            )
            meal_stub.setFillColor(colors.red)
            meal_stub.setFont("Night-Driver", 15)
            meal_stub.drawString(
                x = inch_to_points((8.3 / 2) + 1.73),
                y = inch_to_points(i + 0.65),
                text = "BALIK-LANTAW"
            )
            meal_stub.setFillColor(colors.red)
            meal_stub.setFont("Night-Driver", 12)
            meal_stub.drawString(
                x = inch_to_points((8.3 / 2) + 2.15),
                y = inch_to_points(i + 0.4),
                text = "2026"
            )
    except Exception as e:
        print(f"{type(e).__name__}: Error in creating PDF. {e}")
    else:
        meal_stub.save()

# For debugging purposes
create_A4_batch(
    ids = data[0:26]['unencrypted_id'].tolist()
)

In [8]:
data[0:26]['unencrypted_id'].tolist()

['MT_00001',
 'MT_00002',
 'MT_00003',
 'MT_00004',
 'MT_00005',
 'MT_00006',
 'MT_00007',
 'MT_00008',
 'MT_00009',
 'MT_00010',
 'MT_00011',
 'MT_00012',
 'MT_00013',
 'MT_00014',
 'MT_00015',
 'MT_00016',
 'MT_00017',
 'MT_00018',
 'MT_00019',
 'MT_00020',
 'MT_00021',
 'MT_00022',
 'MT_00023',
 'MT_00024',
 'MT_00025',
 'MT_00026']

In [7]:
## Mass generation of PDFs

N = 26

# Loop through the DataFrame in chunks of n rows
for i in tqdm(range(0, len(data), N), desc="Generating PDF files"):
    chunk = data[i:i+N]['unencrypted_id']
    ids = chunk.tolist()

    create_A4_batch(ids)
    # # # Split into left and right chunks
    # ids_left = ids[0:13]
    # ids_right = ids[13:26]  # The remaining indices (11 to 21 inclusive)

    # print(ids_left, len(ids_left))
    # print(ids_right, len(ids_right))


In [11]:
try:
    data.to_parquet(r"../data/data.parquet", index=False)
    data.to_csv(r"../data/data.csv", index=False)
except Exception as e:
    print(f"{type(e).__name__}: Error in saving data. {e}")
else:
    print("Saved ['csv', 'parquet'] data to '../data/'")

In [2]:
zip_batch(['qr_codes', 'pdfs', r'../data'])

TypeError: Error in zipping files. 'module' object is not callable. Did you mean: 'tqdm.tqdm(...)'?
